In [1]:
import pandas as pd
from collections import defaultdict
from typing import DefaultDict
from pathlib import Path
import datasets as nlp
import sys

train_dataset  = nlp.load_dataset('wikisql', split=nlp.Split.TRAIN)
valid_dataset = nlp.load_dataset('wikisql', split=nlp.Split.VALIDATION)

Found cached dataset wikisql (C:/Users/krilo/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)
Found cached dataset wikisql (C:/Users/krilo/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)


In [17]:
train = pd.DataFrame(train_dataset)
valid = pd.DataFrame(valid_dataset)

In [19]:
valid[["table", "sql"]]

,table,sql
0,"{'header': ['Player', 'No.', 'Nationality', 'P...",{'human_readable': 'SELECT Position FROM table...
1,"{'header': ['Player', 'No.', 'Nationality', 'P...",{'human_readable': 'SELECT COUNT School/Club T...
2,"{'header': ['Player', 'No.', 'Nationality', 'P...",{'human_readable': 'SELECT School/Club Team FR...
3,"{'header': ['Player', 'No.', 'Nationality', 'P...",{'human_readable': 'SELECT Player FROM table W...
4,"{'header': ['Player', 'No.', 'Nationality', 'P...",{'human_readable': 'SELECT Player FROM table W...
...,...,...
8416,"{'header': ['Date', 'Opponent', 'Score', 'Loss...",{'human_readable': 'SELECT Loss FROM table WHE...
8417,"{'header': ['Date', 'Opponent', 'Score', 'Loss...",{'human_readable': 'SELECT MAX Attendance FROM...
8418,"{'header': ['Representative', 'Years', 'State'...",{'human_readable': 'SELECT Party FROM table WH...
8419,"{'header': ['Representative', 'Years', 'State'...",{'human_readable': 'SELECT State FROM table WH...


In [21]:
valid[["table"]].iloc[1].values

array([{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-11', 'section_title': 'L', 'caption': 'L', 'rows': [['Antonio Lang', '21', 'United States', 'Guard-Forward', '1999-2000', 'Duke'], ['Voshon Lenard', '2', 'United States', 'Guard', '2002-03', 'Minnesota'], ['Martin Lewis', '32, 44', 'United States', 'Guard-Forward', '1996-97', 'Butler CC (KS)'], ['Brad Lohaus', '33', 'United States', 'Forward-Center', '1996', 'Iowa'], ['Art Long', '42', 'United States', 'Forward-Center', '2002-03', 'Cincinnati'], ['John Long', '25', 'United States', 'Guard', '1996-97', 'Detroit'], ['Kyle Lowry', '3', 'United States', 'Guard', '2012-Present', 'Villanova']], 'name': 'table_10015132_11'}],
      dtype=object)

In [3]:
def get_schema(x):
    return ", ".join(train_dataset[0]['table']['header'])


def process_splits(data: pd.DataFrame) -> pd.DataFrame:
    data.loc[:, "query"] = data.apply(lambda x: x["sql"]['human_readable'], axis=1)
    data.loc[:, 'query_schema'] = data.apply(lambda x: get_schema(x), axis=1)
    data.loc[:, "question"] = data.apply(lambda x: f'{x["question"]} | {x["query_schema"]}'.replace('  ', ' '), axis=1)
    data = data.drop(columns=["phase", "table", "query_schema", "sql"], axis=1)
    return data

In [4]:
prepared_pauq_train = process_splits(train)
prepared_pauq_valid = process_splits(valid)

In [5]:
prepared_pauq_train

,question,query
0,Tell me what the notes are for South Australia...,SELECT Notes FROM table WHERE Current slogan =...
1,What is the current series where the new serie...,SELECT Current series FROM table WHERE Notes =...
2,What is the format for South Australia? | Stat...,SELECT Format FROM table WHERE State/territory...
3,Name the background colour for the Australian ...,SELECT Text/background colour FROM table WHERE...
4,how many times is the fuel propulsion is cng? ...,SELECT COUNT Fleet Series (Quantity) FROM tabl...
...,...,...
56350,What time was the match played with a score of...,SELECT Time FROM table WHERE Score = 3-2
56351,On which ground did the team play Aston Villa?...,SELECT Ground FROM table WHERE Opponent = asto...
56352,What kind of competition was it at San Siro at...,SELECT Competition FROM table WHERE Ground = s...
56353,What is the total number of decile for the red...,SELECT COUNT Decile FROM table WHERE Name = re...


In [6]:
import numpy as np
m = []
for i, j in prepared_pauq_train.to_records(index=False):
    tmp = (len(i.split()), len(j.split()))
    m.append(tmp)
m = np.array(m)

In [11]:
prepared_pauq_train.to_records(index=False)

rec.array([('Tell me what the notes are for South Australia | State/territory, Text/background colour, Format, Current slogan, Current series, Notes', 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'),
           ('What is the current series where the new series began in June 2011? | State/territory, Text/background colour, Format, Current slogan, Current series, Notes', 'SELECT Current series FROM table WHERE Notes = New series began in June 2011'),
           ('What is the format for South Australia? | State/territory, Text/background colour, Format, Current slogan, Current series, Notes', 'SELECT Format FROM table WHERE State/territory = South Australia'),
           ...,
           ('What kind of competition was it at San Siro at 18:30 GMT? | State/territory, Text/background colour, Format, Current slogan, Current series, Notes', 'SELECT Competition FROM table WHERE Ground = san siro AND Time = 18:30 gmt'),
           ('What is the total number of decile for the red

In [13]:
for i, j in prepared_pauq_train.to_records(index=False):
    d = dict()
    d['src'] = i.strip()
    d['trg'] = j.strip()
    print(d)
    break

{'src': 'Tell me what the notes are for South Australia | State/territory, Text/background colour, Format, Current slogan, Current series, Notes', 'trg': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'}


In [14]:
import json

def json2jsonl(df, writename):
    with open(writename, 'w', encoding='utf8') as f:
        for i, j in df.to_records(index=False):
            d = dict()
            d['src'] = i.strip()
            d['trg'] = j.strip()
            json.dump(d, f)
            f.write('\n')

In [15]:
json2jsonl(prepared_pauq_train, "train.jsonl")

In [16]:
json2jsonl(prepared_pauq_valid, "valid.jsonl")